In [3]:
import csv
import pandas as pd 
import h5py
import os
import numpy as np
import torch

In [33]:
############# for all

path = '/home/mvasist/simulations/6_params/'  

X=[]
T=[]
for k in range(1, 75): #60, 79
    # if k==79: continue
    dfX_M= pd.read_csv(path+ 'X_1.5Msim_TintLkIRLgLH2OLCH4LCO'+ str(k) + '.csv', engine ='python')  #, low_memory = False)
    dfT_M= pd.read_csv(path+ 'T_1.5Msim_TintLkIRLgLH2OLCH4LCO'+ str(k) + '.csv', engine ='python')  #, low_memory = False)
    # print(len(dfX_M.values), k)
    X.append(dfX_M.values) 
    T.append(dfT_M.values)

comb_np_array_X = np.vstack(X)
comb_np_array_T = np.vstack(T)

x = torch.from_numpy(comb_np_array_X[:,1:])
t = torch.from_numpy(comb_np_array_T[:,1:])

xt=[]
for i in range(len(x)):
    xt.append(torch.cat([t[i],x[i]]))
xt = torch.cat(xt)
xt = xt.reshape(len(x),int(len(xt)/len(x)))

xt_shuffled=[]
for j in range(len(x)):
    r=np.random.randint(0,len(x))
    if r==j:
        r= np.random.randint(0,len(x))
    xt_shuffled.append(torch.cat([t[j],x[r]]))
xt_shuffled = torch.cat(xt_shuffled)
xt_shuffled = xt_shuffled.reshape(len(x),int(len(xt_shuffled)/len(x)))

XT = torch.cat((xt,xt_shuffled),axis=0)
labels = torch.cat((torch.ones(len(x),1),torch.zeros(len(x),1)))

sample = {'target': labels, 'input': XT }


In [34]:
len(XT) #, labels #= XT[:10000], labels[:10000]

221852

In [35]:
labels

tensor([[1.],
        [1.],
        [1.],
        ...,
        [0.],
        [0.],
        [0.]])

In [5]:
# XT, labels = XT[:1000], labels[:1000] 

In [36]:
with h5py.File('6params_train_bce1M2.h5', 'w') as f: 
    f.create_dataset('data', data= XT, dtype=np.float32)
    f.create_dataset('labels', data=labels, dtype=np.float32)
f.close()

In [37]:
with h5py.File('6params_train_bce1M2.h5', 'r') as f: 
    d = np.array(f.get('data'))
    l = np.array(f.get('labels'))
f.close()

In [38]:
len(d)

221852